In [10]:
!pip install selenium
!pip install splinter
!pip install Flask-PyMongo

  Using cached Flask_PyMongo-2.3.0-py2.py3-none-any.whl (12 kB)


In [4]:
import time
from selenium import webdriver
driver = webdriver.Chrome() 
#"C:/Users/jfreese/Anaconda3/chromedriver"
driver.get('http://www.google.com/');

time.sleep(5)

search_box = driver.find_element_by_name('q')

search_box.send_keys('ChromeDriver')

search_box.submit()

time.sleep(5) # Let the user actually see something!

driver.quit()

In [5]:
! which chromedriver

/c/Users/jfreese/Anaconda3/chromedriver


In [11]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [12]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [13]:
news_url = 'https://redplanetscience.com/'
browser.visit( news_url)

# Create HTML object
html = browser.html

# Parse HTML with BeautifulSoup
soup = bs( html, 'html.parser')

In [27]:
title = soup.find_all( 'div', class_ = 'content_title')[0].text

# Retrieve the latest news paragraph
paragraph = soup.find_all( 'div', class_ = 'article_teaser_body')[0].text
print( title)
print( paragraph)

IndexError: list index out of range

In [15]:
space_images_url = 'https://spaceimages-mars.com/'
browser.visit( space_images_url)

# Create HTML object
html = browser.html

# Parse HTML with BeautifulSoup
soup = bs( html, 'html.parser')

In [16]:
featured_image = soup.find_all( 'img', class_ = 'headerimage fade-in')[0]['src']
featured_image_url = space_images_url + featured_image
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


In [17]:
facts_url = 'https://galaxyfacts-mars.com/'

facts_table = pd.read_html(facts_url, header=0)
facts_table[0]

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


In [18]:
facts_table = pd.read_html(facts_url)
facts_df = facts_table[1]
facts_df.columns =['Description', 'Value']
facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [19]:
facts_html = facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
  

In [20]:
hemispheres_url = 'https://marshemispheres.com/'
browser.visit( hemispheres_url)

# Create HTML object
html = browser.html

# Parse HTML with BeautifulSoup
soup = bs( html, 'html.parser')

In [21]:
items = soup.find_all('div', class_='item')

mars_hemi_urls = []
mars_hemi_titles = []

for item in items:
    mars_hemi_urls.append( hemispheres_url + item.find('a')['href'])
    mars_hemi_titles.append( item.find('h3').text.strip())

print( mars_hemi_urls)
mars_hemi_titles

['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']


['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [22]:
hemi_img_urls = []

for url in mars_hemi_urls:
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Find image urls and append to list
    source_url = hemispheres_url + soup.find('img',class_='wide-image')['src']
    hemi_img_urls.append( source_url)
    
hemi_img_urls

['https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [23]:
hemi_img_list = []

for i in range( len( mars_hemi_titles)):
    hemi_img_list.append({ 'title':mars_hemi_titles[i], 'img_url':hemi_img_urls[i]})

hemi_img_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [24]:
mars_dict = {
    "news_title":title,
    "news_p":paragraph,
    "featured_image_url":featured_image_url,
    "fact_table":facts_html,
    "hemisphere_images":hemi_img_list
}

In [25]:
mars_dict

{'news_title': "All About the Laser (and Microphone) Atop Mars 2020, NASA's Next Rover",
 'news_p': 'SuperCam is a rock-vaporizing instrument that will help scientists hunt for Mars fossils.',
 'featured_image_url': 'https://spaceimages-mars.com/image/featured/mars3.jpg',
 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    

In [26]:
browser.quit()